# Cleaning

In [89]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [90]:
data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv")

data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_words
0,0000997932d777bf,explanation\n edits made username hardcore ...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '..."
1,000103f0d9cfb60f,daww matches background colour im seemingly ...,0,0,0,0,0,0,"['daww', 'matches', 'background', 'colour', 'i..."
2,000113f07ec002fd,hey man im really trying edit war guy c...,0,0,0,0,0,0,"['hey', 'man', 'im', 'really', 'trying', 'edit..."
3,0001b41b1c6bb37e,cant make real suggestions improvement won...,0,0,0,0,0,0,"['cant', 'make', 'real', 'suggestions', 'impro..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page', ..."


# Text Vectorization Layer
### Following [https://www.tensorflow.org/text/tutorials/text_classification_rnn](http://)

In [91]:
NUM_ROWS = 10000

MAX_LENGTH = None
encoder = tf.keras.layers.TextVectorization(output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'article', 'page', 'wikipedia', 'would', 'talk',
       'like', 'one', 'please', 'dont', 'see', 'also', 'im', 'know',
       'think', 'edit', 'people', 'use', 'articles'], dtype='<U322')

### Building the model

In [92]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

regularization_layer = tf.keras.layers.Dense(
    64, 
    activation="relu", 
    #kernel runs before activation, activity runs after
    kernel_regularizer=tf.keras.regularizers.l1(0.001),
    activity_regularizer=tf.keras.regularizers.l2(0.001)
)

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [93]:
binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = binaryDf.iloc[:split_cutoff]
validation_data = binaryDf.iloc[split_cutoff:]

training_target = training_data.pop("toxic")
validation_target = validation_data.pop("toxic")

In [94]:
# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=2)

history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback])

Epoch 1/10
250/250 [==============================] - 47s 116ms/step - loss: 0.4074 - accuracy: 0.9045 - val_loss: 0.2426 - val_accuracy: 0.9125
Epoch 2/10
250/250 [==============================] - 23s 90ms/step - loss: 0.1876 - accuracy: 0.9175 - val_loss: 0.1728 - val_accuracy: 0.9360
Epoch 3/10
250/250 [==============================] - 23s 91ms/step - loss: 0.0758 - accuracy: 0.9732 - val_loss: 0.1875 - val_accuracy: 0.9470
Epoch 4/10
250/250 [==============================] - 23s 91ms/step - loss: 0.0238 - accuracy: 0.9934 - val_loss: 0.2344 - val_accuracy: 0.9460
Epoch 5/10
250/250 [==============================] - 22s 89ms/step - loss: 0.0090 - accuracy: 0.9995 - val_loss: 0.2479 - val_accuracy: 0.9470
Epoch 6/10
250/250 [==============================] - 23s 92ms/step - loss: 0.0049 - accuracy: 0.9996 - val_loss: 0.2873 - val_accuracy: 0.9475
Epoch 7/10
250/250 [==============================] - 23s 91ms/step - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.3088 - val_accuracy

### Getting test data

In [95]:
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels.loc[test_labels["toxic"] >= 0]
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")

In [96]:
# Tests all rows with a value of 0 or 1

test_df = merged_df[["comment_text", "toxic"]]
testTarget = test_df.pop("toxic")
model.evaluate(test_df, testTarget)

2000/2000 [==============================] - 71s 35ms/step - loss: 0.8803 - accuracy: 0.8207


[0.8803300261497498, 0.8207352757453918]

In [98]:
# Tests based on validation data

model.evaluate(validation_data, validation_target)

63/63 [==============================] - 2s 25ms/step - loss: 0.3147 - accuracy: 0.9470


[0.31466495990753174, 0.9470000267028809]

In [97]:
# Tests only rows with a toxic value of 1

test_df = merged_df[["comment_text", "toxic"]]
newTest_df = test_df.loc[test_df["toxic"] == 1]

newTestTarget = newTest_df.pop("toxic")
model.evaluate(newTest_df, newTestTarget)

191/191 [==============================] - 6s 29ms/step - loss: 1.0610 - accuracy: 0.7760


[1.0610125064849854, 0.7760262489318848]